# Import Libraries

In [43]:
import pandas as pd
from operator import itemgetter
import re, math

In [3]:
def flatten(l):
    return [item for sublist in l for item in sublist]

# Read Data

In [4]:
data = pd.read_csv('tfidf_dataset.csv')
# data = pd.read_csv('gs://dataproc-staging-us-east1-1019160201255-embclvt4/notebooks/jupyter/tfidf_dataset.csv')

g:\Softwares\Python\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (2,3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
# List of Stop Words
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [7]:
# Taken from output of Hive Queries (Top 10 Videos of Spam and Ham Comments)
spam_video_id = ['4MkC65emkG4',
'#NAME? ',
'4tR_H21CuP8',
'a7sf_H2cFdM',
'ne4CnyNW904',
'Q07Cp6tswnQ',
'W4SKO5jU641',
'X_PZvUM8P_U',
'J4qvZiJegly',
'AzWiUQEdJoo']

ham_video_id = ['#NAME?',
'Q0CbN8sfihy',
'LunHybOKIJU,'
'tзAVtQkEHAE',
'19-DM9uBtVI',
't8sgy@faXyg',
'QeBCulZSORS',
'doudRCrK80E',
'uE6105HiJts',
'YL9uHq4HsQI']

In [8]:
data.columns

Index(['video_id', 'comment_text', 'likes', 'replies', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6'],
      dtype='object')

# Data Cleaning

In [9]:
data.drop(columns=['likes', 'replies', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], inplace=True)

In [10]:
data['cleaned_data'] = data['comment_text'].str.strip()

In [11]:
# Get Comments Only (Remove Numbers, Emojis Punctuations, Symbols)
data['cleaned_data'] = data['cleaned_data'].map(lambda x: re.sub("[^a-z]", " ",  str(x).lower().strip()))

In [12]:
# Remove Stopwords
data['cleaned_data'] = data['cleaned_data'].map(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [13]:
# Remove Words/Letters less than equal to length 2
data['cleaned_data'] = data['cleaned_data'].map(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))

In [14]:
data = data[data['cleaned_data']!='']

In [15]:
data.head()

,video_id,comment_text,cleaned_data
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,accurate call price closer calling
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,samsung phone
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",thank gosh place watch without speed support
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,happened home button iphone cough copying sams...
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,power disease care cure keep caring others bes...


In [16]:
# Make list of all Ham and Spam Words
spam_doc = data.query(f"""video_id in {spam_video_id}""")['cleaned_data'].to_list()
ham_doc = data.query(f"""video_id in {ham_video_id}""")['cleaned_data'].to_list()

# Flatten List (Remove and Lists present in Lists)
spam_doc = flatten([i.split(' ') for i in spam_doc])
ham_doc = flatten([i.split(' ') for i in ham_doc])

In [17]:
# Make Union of Ham and Spam
uniqueWords = list(set(ham_doc).union(set(spam_doc)))

# Flatten Lists
uniqueWords = flatten([i.split(' ') for i in uniqueWords])

len(uniqueWords)

10722

In [18]:
# Make Dictionary of Unique Ham and Spam Keywords
num_ham_doc = dict.fromkeys(uniqueWords, 0)
num_spam_doc = dict.fromkeys(uniqueWords, 0)

# Count the Frequencies of the Words Present in the list of Ham Words List
for word in ham_doc:
        num_ham_doc[word] += 1    

# Count the Frequencies of the Words Present in the list of Spam Words List
for word in spam_doc:
    num_spam_doc[word] += 1

# Term Frequency (TF)

In [19]:
# Count the number of times a word appears in a document divded by the total number of words in the document.
def computeTermFrequency(wordDict, comments_corpus):
    tfidf_dict = {}
    comments_corpusCount = len(comments_corpus)
    for word, count in wordDict.items():
        tfidf_dict[word] = count / float(comments_corpusCount)
    return tfidf_dict

In [20]:
tf_ham = computeTermFrequency(num_ham_doc, ham_doc)
tf_spam = computeTermFrequency(num_spam_doc, spam_doc)

In [39]:
# tf_ham
dict(sorted(tf_ham.items(), key = itemgetter(1), reverse = True)[:10])

{'like': 0.013634072028232613,
 'love': 0.010269393481072314,
 'video': 0.007660945617149997,
 'really': 0.0063238420902654476,
 'one': 0.006181363845597422,
 'much': 0.0058306481664145895,
 'get': 0.0051292168080489245,
 'know': 0.004953858968457509,
 'make': 0.0048552201836873365,
 'food': 0.004811380723789483}

In [40]:
# tf_spam
dict(sorted(tf_spam.items(), key = itemgetter(1), reverse = True)[:10])

{'people': 0.012972382480456082,
 'money': 0.01218721196190216,
 'much': 0.008261359369132557,
 'bear': 0.007885843034171986,
 'like': 0.007476188850578636,
 'get': 0.00692998327245417,
 'think': 0.0067251561806574945,
 'congrats': 0.006349639845696924,
 'one': 0.00611067490526747,
 'good': 0.005530331478510224}

# Inverse Data Frequency (IDF)

In [41]:
# Calculate IDF for Ham and Spam Dataset
def InverseDocumentFrequency(documents):
    N = len(documents)
    
    idf_dict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idf_dict[word] += 1
    
    for word, val in idf_dict.items():
        idf_dict[word] = math.log(N / float(val))
    return idf_dict

In [46]:
idfs = InverseDocumentFrequency([num_ham_doc, num_spam_doc])

In [51]:
dict(sorted(idfs.items(), key = itemgetter(1), reverse = True)[:10])

{'suarez': 0.6931471805599453,
 'draw': 0.6931471805599453,
 'nin': 0.6931471805599453,
 'mud': 0.6931471805599453,
 'har': 0.6931471805599453,
 'whoop': 0.6931471805599453,
 'charlie': 0.6931471805599453,
 'trendingggg': 0.6931471805599453,
 'defo': 0.6931471805599453,
 'merit': 0.6931471805599453}

In [52]:
# Calculate Term Frequency Inverse Document Frequency
def TermFrequencyInvDocFreq(comments_corpus, idfs):
    tfidf = {}
    for word, val in comments_corpus.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [53]:
tfidf_ham = TermFrequencyInvDocFreq(tf_ham, idfs)
tfidf_spam = TermFrequencyInvDocFreq(tf_spam, idfs)

df = pd.DataFrame([tfidf_ham, tfidf_spam])

In [54]:
pd.set_option('display.max_rows', 1000)

In [55]:
# Top 10 HAM
dict(sorted(tfidf_ham.items(), key = itemgetter(1), reverse = True)[:10])

{'enzo': 0.0022182654558591877,
 'vegan': 0.0021043134632636818,
 'fergie': 0.0017700542849835301,
 'autumn': 0.0014585855052224796,
 'danny': 0.0014509887057161128,
 'ayesha': 0.0009192127402704169,
 'pepper': 0.0008508415447131131,
 'lmao': 0.0008356479457003789,
 'foot': 0.0007976639481685436,
 'amelia': 0.0007065023540921386}

In [56]:
# Top 10 Spam
dict(sorted(tfidf_spam.items(), key = itemgetter(1), reverse = True)[:10])

{'polar': 0.0036913583507101172,
 'zombie': 0.0025792183347910433,
 'thrones': 0.001656378747113514,
 'marques': 0.0011121400159190737,
 'mkbhd': 0.0010411523553284945,
 'donated': 0.0009228395876775293,
 'puzzle': 0.0008755144806171432,
 'toby': 0.0008755144806171432,
 'mtv': 0.0008518519270869502,
 'earnings': 0.0007335391594359849}

In [57]:
df.T

,0,1
makes,0.000000,0.000000
suarez,0.000008,0.000000
draw,0.000000,0.000024
subscribed,0.000000,0.000000
huge,0.000000,0.000000
...,...,...
nhope,0.000030,0.000000
reborn,0.000008,0.000000
hair,0.000000,0.000000
comparing,0.000015,0.000000
